In [ ]:
import tensorflow as tf
import matplotlib.image as img
%matplotlib inline
import numpy as np
from collections import defaultdict
import collections
from shutil import copy  
from shutil import copytree, rmtree
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import os
import random
from tensorflow.keras import models
import seaborn as sns
import cv2

In [ ]:
import pandas as pd
import numpy as np
import math
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load DGA From Netlab 360

In [ ]:
columns_dga = ["domain_name","start_time", "end_time","dga_family"]
Netlab_dga = pd.read_csv('/content/drive/My Drive/Colab Notebooks/netlab360.csv', names=columns_dga)
Netlab_dga.drop(["start_time", "end_time"], axis=1, inplace=True)
Netlab_dga.head()

,domain_name,dga_family
0,odqzesdvd.info,nymaim
1,mcrwnxdzu.info,nymaim
2,poxrscvfoy.biz,nymaim
3,dupzwi.info,nymaim
4,pgldibr.com,nymaim


In [ ]:
Netlab_dga.shape

(1048576, 2)

In [ ]:
Netlab_dga.isnull().sum()

domain_name    0
dga_family     0
dtype: int64

In [ ]:
columns_dga = ["domain_name","start_time", "end_time","dga_family"]
Netlab_dga2 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/netlab360_2.csv', names=columns_dga)
Netlab_dga2.drop(["start_time", "end_time"], axis=1, inplace=True)
Netlab_dga2.head()

,domain_name,dga_family
0,btausagabardinedazyx.com,banjori
1,omgysagabardinedazyx.com,banjori
2,fpdcsagabardinedazyx.com,banjori
3,aqvzsagabardinedazyx.com,banjori
4,skfasagabardinedazyx.com,banjori


In [ ]:
Netlab_dga2.shape

(440019, 2)

In [ ]:
#Missing or NAN values in dataset
Netlab_dga2.isnull().sum()

domain_name    0
dga_family     0
dtype: int64

In [ ]:
# Concatenating both datasets
Netlab_dga_data = pd.concat([Netlab_dga, Netlab_dga2])

print("Whole Dataset has {} rows".format(Netlab_dga_data.shape[0]))
Netlab_dga_data.head()

Whole Dataset has 1488595 rows


,domain_name,dga_family
0,odqzesdvd.info,nymaim
1,mcrwnxdzu.info,nymaim
2,poxrscvfoy.biz,nymaim
3,dupzwi.info,nymaim
4,pgldibr.com,nymaim


## Load top 1M Alexa Legitimate domain

In [ ]:
Alexa_legit_columns = ["domain_name"]
Alexa_legit_domains = pd.read_csv('/content/drive/My Drive/Colab Notebooks/top-1m.csv', names=Alexa_legit_columns)
Alexa_legit_domains["label"] = "legit"
Alexa_legit_domains.head()

,domain_name,label
1,google.com,legit
2,youtube.com,legit
3,tmall.com,legit
4,baidu.com,legit
5,qq.com,legit


In [ ]:
Alexa_legit_domains.shape

(613032, 2)

## Concatinate both legitmate & DGA domains

In [ ]:
# Concatenating both datasets
Alexa_legit_domains.rename(columns = {'label':'dga_family'}, inplace = True)
Domain_data = pd.concat([Netlab_dga_data, Alexa_legit_domains])

print("Whole Dataset has {} rows".format(Domain_data.shape[0]))
Domain_data.head()

Whole Dataset has 2101627 rows


,domain_name,dga_family
0,odqzesdvd.info,nymaim
1,mcrwnxdzu.info,nymaim
2,poxrscvfoy.biz,nymaim
3,dupzwi.info,nymaim
4,pgldibr.com,nymaim


## Dealing with Null and duplicate value 

In [ ]:
Domain_data.isnull().sum()

domain_name    0
dga_family     0
dtype: int64

In [ ]:
Domain_data.duplicated().sum()

24989

In [ ]:
new_output = Domain_data[Domain_data.duplicated('domain_name')]
print("Duplicated values",new_output)

Duplicated values              domain_name dga_family
34634         pocjubt.ru     nymaim
34635     qgvhlpwrjjo.ru     nymaim
34637        fsonck.info     nymaim
34638       wsfliiyg.com     nymaim
34639   tlffbudictx.info     nymaim
...                  ...        ...
428556      purymuq.info      simda
428557      hahogaz.info      simda
428559      masosom.info      simda
428564      pupygop.info      simda
428568      puvytuv.info      simda

[24989 rows x 2 columns]


In [ ]:
Domain_data = Domain_data.drop_duplicates(subset=['domain_name'])

In [ ]:
Domain_data.duplicated().sum()

0

In [ ]:
Domain_data.shape

(2076638, 2)

In [ ]:
grouped=Domain_data['domain_name'].groupby(Domain_data['dga_family'])
grouped.count()

dga_family
abcbot                27
antavmu               16
bamital              104
banjori           469896
bigviktor           1000
blackhole              2
ccleaner               1
chinad              1000
conficker            492
copperstealer         11
cryptolocker        1000
dircrypt             736
dyre                1000
emotet            500540
enviserv             492
feodo                247
flubot             29999
fobber_v1            298
fobber_v2            299
gameover           12000
gspy                 100
kfos                 124
legit             613032
locky               1158
madmax                 1
matsnu               908
mirai                  1
murofet             8560
mydoom             10047
necro               2708
necurs              8190
ngioweb             5274
nymaim               333
omexo                 40
padcrypt             168
proslikefan          100
pykspa_v1          44675
pykspa_v2_fake       800
pykspa_v2_real       199
qadars        

In [ ]:
Domain_data['dga_family'].nunique()

57

### Grouping family

In [ ]:
Domain_data[Domain_data["dga_family"] == "tordwm"].head()

,domain_name,dga_family
596524,a3698d83.top,tordwm
596525,3d0d1820.top,tordwm
596526,adb205b1.top,tordwm
596527,bfdb1290.top,tordwm
596528,4d67ecaf.top,tordwm


In [ ]:
Domain_data['dga_family']=Domain_data['dga_family'].replace(['legit'], "Super_family_1")
Domain_data['dga_family'] = Domain_data['dga_family'].replace(['abcbot', 'banjori','blackhole','xshellghost','virut','vidro','vawtrak','tofsee','tinba','tempedreve','symmi','simda','shifu','ranbyus','ramnit','pykspa_v2_real','pykspa_v2_fake','pykspa_v1','proslikefan','padcrypt','nymaim','necurs','necro','mydoom','murofet','mirai','locky','fobber_v2','fobber_v1','flubot','feodo','emotet','dyre','dircrypt','cryptolocker','conficker','blackhole','banjori','madmax'],"Super_family_2")
Domain_data['dga_family']=Domain_data['dga_family'].replace(['shiotob', 'rovnix','qadars','gameover','chinad','ccleaner','tinynuke'], "Super_family_3")
Domain_data['dga_family']=Domain_data['dga_family'].replace(['omexo', 'gspy','enviserv','copperstealer','bamital','antavmu','tordwm'], "Super_family_4")
Domain_data['dga_family']=Domain_data['dga_family'].replace(['ngioweb', 'matsnu','bigviktor','suppobox'], "Super_family_5")
Domain_data['dga_family']=Domain_data['dga_family'].replace(['kfos'],"Super_family_6")

In [ ]:
Domain_data.head()

,domain_name,dga_family
0,odqzesdvd.info,Super_family_2
1,mcrwnxdzu.info,Super_family_2
2,poxrscvfoy.biz,Super_family_2
3,dupzwi.info,Super_family_2
4,pgldibr.com,Super_family_2


In [ ]:
Domain_data.nunique()

domain_name    2076638
dga_family           6
dtype: int64

In [ ]:
grouped=Domain_data['domain_name'].groupby(Domain_data['dga_family'])
grouped.count()

dga_family
Super_family_1     613032
Super_family_2    1251686
Super_family_3     201043
Super_family_4       1273
Super_family_5       9480
Super_family_6        124
Name: domain_name, dtype: int64

## Feature engineering

In [ ]:
! pip install tldextract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 93 kB 1.4 MB/s 


In [ ]:
def entropy(domain_name):
    """ Function which computes the entropy of a given domain name based on it's chars """
    elements, length = Counter(domain_name), len(domain_name)    
    
    return -sum(element/length * math.log(element/length, 2) for element in elements.values())

In [ ]:
import tldextract
def get_domain_name(domain):
    """ Function which extracts domain name from subdomain name """
    res = tldextract.extract(domain)
    return res.domain if len(res.domain) > len(res.subdomain) or entropy(res.domain) > entropy(res.subdomain) else res.subdomain


Domain_data["domain"] = Domain_data["domain_name"].apply(lambda domain: get_domain_name(domain))

In [ ]:
def firstDigitIndex( s ):
    for i, c in enumerate(s):
        if c.isdigit():
            return i + 1
    return 0

In [ ]:
def H_entropy (x):
    # Calculate Shannon Entropy
    prob = [ float(x.count(c)) / len(x) for c in dict.fromkeys(list(x)) ] 
    H = - sum([ p * np.log2(p) for p in prob ]) 
    return H

In [ ]:
def vowel_character_ratio (x):
    # Calculate vowel ratio
    x = x.lower()
    vowels_pattern = re.compile('([aeiou])')
    vowels = re.findall(vowels_pattern, x)
    try:
        ratio = len(vowels) / len(x)
    except: # catch zero devision exception 
        ratio = 0  
    return ratio

In [ ]:
def consonant_character_ratio (x):
    # Calculate consonant  ratio
    x = x.lower()
    consonants_pattern = re.compile('([b-df-hj-np-tv-z])')
    consonants = re.findall(consonants_pattern, x)
    try:
        ratio = len(consonants) / len(x)
    except: # catch zero devision exception 
        ratio = 0  
    return ratio

In [ ]:
def get_domain_level_number(domain_name):
  level= domain_name.split('.')
  return len(level)

In [ ]:
def vowel_consonant_ratio (x):
    # Calculate vowel to consonant ratio
    x = x.lower()
    vowels_pattern = re.compile('([aeiou])')
    consonants_pattern = re.compile('([b-df-hj-np-tv-z])')
    vowels = re.findall(vowels_pattern, x)
    consonants = re.findall(consonants_pattern, x)
    try:
        ratio = len(vowels) / len(consonants)
    except: # catch zero devision exception 
        ratio = 0  
    return ratio

In [ ]:
import re
# Applying functions above
Domain_data["length"] = Domain_data["domain"].apply(lambda domain: len(domain))
Domain_data["entropy"] = Domain_data["domain"].apply(lambda domain: entropy(domain))
Domain_data["first_Digit_Index"] = Domain_data["domain_name"].apply(lambda domain: firstDigitIndex(domain))
Domain_data["H_entropy"] = Domain_data["domain_name"].apply(lambda domain: H_entropy(domain))
Domain_data["vowel_consonant_ratio"] = Domain_data["domain_name"].apply(lambda domain: vowel_consonant_ratio(domain))
Domain_data["vowel_character_ratio"] = Domain_data["domain_name"].apply(lambda domain: vowel_character_ratio(domain))
Domain_data["consonant_character_ratio"] = Domain_data["domain_name"].apply(lambda domain: consonant_character_ratio(domain))
Domain_data["level_no"] = Domain_data["domain_name"].apply(lambda domain: get_domain_level_number(domain))
Domain_data.head()

,domain_name,dga_family,domain,length,entropy,first_Digit_Index,H_entropy,vowel_consonant_ratio,vowel_character_ratio,consonant_character_ratio,level_no
0,odqzesdvd.info,Super_family_2,odqzesdvd,9,2.641604,0,3.324863,0.444444,0.285714,0.642857,2
1,mcrwnxdzu.info,Super_family_2,mcrwnxdzu,9,3.169925,0,3.664498,0.300000,0.214286,0.714286,2
2,poxrscvfoy.biz,Super_family_2,poxrscvfoy,10,3.121928,0,3.664498,0.300000,0.214286,0.714286,2
3,dupzwi.info,Super_family_2,dupzwi,6,2.584963,0,3.277613,0.666667,0.363636,0.545455,2
4,pgldibr.com,Super_family_2,pgldibr,7,2.807355,0,3.459432,0.250000,0.181818,0.727273,2


In [ ]:
# A choice of 3 to 5 grams seems okay
import sklearn.feature_extraction

split_condition = Domain_data["dga_family"] == "Super_family_1"
legit = Domain_data[split_condition]
dga = Domain_data[~split_condition]

alexa_vc = sklearn.feature_extraction.text.CountVectorizer(analyzer="char", ngram_range=(3,5), min_df=0.00001, max_df=1.0)
counts_matrix = alexa_vc.fit_transform(legit["domain"])

counts_matrix

<613032x184077 sparse matrix of type '<class 'numpy.int64'>'
	with 11887482 stored elements in Compressed Sparse Row format>

In [ ]:
alexa_counts = np.log10(np.asarray(counts_matrix.sum(axis=0)).flatten())
ngrams_list = alexa_vc.get_feature_names()
print(ngrams_list[100:200])

['--it', '--itb', '--j', '--j1', '--j1a', '--k', '--l', '--l3', '--l3c', '--m', '--m1', '--mg', '--mgb', '--n', '--o', '--p', '--q', '--r', '--s', '--t', '--th', '--w', '--y', '--ym', '--ymc', '--z', '-0-', '-01', '-1-', '-10', '-100', '-101', '-108', '-11', '-116', '-12', '-123', '-125', '-125-', '-12c', '-12ca', '-12cl', '-13', '-14', '-15', '-16', '-164', '-165', '-168', '-168-', '-17', '-18', '-180', '-180-', '-19', '-1x', '-1x2', '-2-', '-20', '-200', '-2001', '-201', '-202', '-2020', '-2021', '-2022', '-204', '-21', '-21-', '-22', '-22c', '-23', '-24', '-24-', '-240', '-25', '-28', '-3-', '-31', '-34', '-36', '-36-', '-360', '-365', '-37', '-37-', '-38', '-38-', '-3d', '-4-', '-40', '-41', '-42', '-42c', '-44', '-448', '-448a', '-48', '-49', '-4k']


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
sorted_ngrams = sorted(zip(ngrams_list, alexa_counts), key=lambda alexa_count: alexa_count[1], reverse=True)
print("Alexa NGrams {}".format(len(sorted_ngrams)))
for ngram, count in sorted_ngrams[:10]:
    print(ngram, count)

Alexa NGrams 184077
ing 4.3406621988556084
ion 4.188253327026504
ent 4.180813775754397
ine 4.176467484599134
the 4.155123393710713
ter 4.151951788707265
lin 4.123198075031999
and 4.120047194353041
ers 4.074048069062633
tor 4.0604334067027406


In [ ]:
Domain_data["alexa_grams"] = alexa_counts * alexa_vc.transform(Domain_data["domain"]).T
Domain_data[Domain_data["dga_family"] == "Super_family_1"].head()  

,domain_name,dga_family,domain,length,entropy,first_Digit_Index,H_entropy,vowel_consonant_ratio,vowel_character_ratio,consonant_character_ratio,level_no,alexa_grams
1,google.com,Super_family_1,google,6,1.918296,0,2.646439,0.800000,0.400000,0.500000,2,24.600344
2,youtube.com,Super_family_1,youtube,7,2.521641,0,3.095795,1.000000,0.454545,0.454545,2,31.696975
3,tmall.com,Super_family_1,tmall,5,1.921928,0,2.725481,0.333333,0.222222,0.666667,2,16.423631
4,baidu.com,Super_family_1,baidu,5,2.321928,0,3.169925,1.000000,0.444444,0.444444,2,12.225260
5,qq.com,Super_family_1,qq,2,-0.000000,0,2.251629,0.250000,0.166667,0.666667,2,0.000000


In [ ]:
!pip install textfeatures

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
words_df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/words.txt", names=["word"],
                             encoding="utf-8", header=None, dtype={"word": np.str})
words_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


,word
0,2
1,1080
2,&c
3,10-point
4,10th


In [ ]:
def clean_words_df(word):
    return str(word).strip().lower()

def keep_alphanumeric(word):
    return str(word).isalpha()



words_df = words_df[words_df["word"].map(lambda word: str(word).isalpha())]
words_df = words_df.applymap(lambda word: str(word).strip().lower())

words_df = words_df.dropna()
words_df = words_df.drop_duplicates()

In [ ]:
dict_cv = sklearn.feature_extraction.text.CountVectorizer(analyzer="char", ngram_range=(3,5), min_df=0.00001, max_df=1.0)
words_counts_matrix = dict_cv.fit_transform(words_df["word"])

dict_counts = np.log(np.asarray(words_counts_matrix.sum(axis=0)).flatten())
words_ngrams_list = dict_cv.get_feature_names()

print(words_ngrams_list[100:200])

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['abbed', 'abber', 'abbes', 'abbet', 'abbey', 'abbi', 'abbie', 'abbil', 'abbin', 'abbis', 'abbit', 'abbl', 'abble', 'abbli', 'abbo', 'abbot', 'abbr', 'abbre', 'abbro', 'abby', 'abc', 'abd', 'abda', 'abde', 'abdi', 'abdic', 'abdit', 'abdo', 'abdoc', 'abdom', 'abdop', 'abdos', 'abdu', 'abduc', 'abe', 'abea', 'abeat', 'abec', 'abece', 'abecu', 'abed', 'abee', 'abef', 'abei', 'abel', 'abela', 'abele', 'abeli', 'abell', 'abels', 'abem', 'abema', 'aben', 'abena', 'abend', 'aber', 'aberd', 'aberg', 'abern', 'aberr', 'abers', 'abes', 'abet', 'abete', 'abeth', 'abeti', 'abeto', 'abett', 'abey', 'abeya', 'abez', 'abf', 'abh', 'abha', 'abho', 'abhor', 'abi', 'abia', 'abial', 'abian', 'abiat', 'abib', 'abic', 'abica', 'abich', 'abici', 'abid', 'abida', 'abide', 'abidi', 'abie', 'abies', 'abiet', 'abif', 'abifi', 'abify', 'abig', 'abiga', 'abil', 'abila']


In [ ]:
Domain_data["words_grams"] = dict_counts * dict_cv.transform(Domain_data["domain"]).T
Domain_data.head()

,domain_name,dga_family,domain,length,entropy,first_Digit_Index,H_entropy,vowel_consonant_ratio,vowel_character_ratio,consonant_character_ratio,level_no,alexa_grams,words_grams
0,odqzesdvd.info,Super_family_2,odqzesdvd,9,2.641604,0,3.324863,0.444444,0.285714,0.642857,2,8.618689,10.200253
1,mcrwnxdzu.info,Super_family_2,mcrwnxdzu,9,3.169925,0,3.664498,0.300000,0.214286,0.714286,2,5.331953,4.442651
2,poxrscvfoy.biz,Super_family_2,poxrscvfoy,10,3.121928,0,3.664498,0.300000,0.214286,0.714286,2,9.758886,11.231941
3,dupzwi.info,Super_family_2,dupzwi,6,2.584963,0,3.277613,0.666667,0.363636,0.545455,2,3.861893,7.928046
4,pgldibr.com,Super_family_2,pgldibr,7,2.807355,0,3.459432,0.250000,0.181818,0.727273,2,12.146719,20.785039


In [ ]:
Domain_data[Domain_data["dga_family"] == "legit"].head(10)

,domain_name,dga_family,domain,length,entropy,first_Digit_Index,H_entropy,vowel_consonant_ratio,vowel_character_ratio,consonant_character_ratio,level_no,alexa_grams,words_grams


In [ ]:
Domain_data.to_csv(r'/content/drive/My Drive/Colab Notebooks/Super_MLDOMAIN_MCLS2.csv')